In [22]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential

In [23]:
df_1=pd.read_csv('dc_part.csv')
df_2=pd.read_csv('chenqin5%.csv')
df_4=pd.read_csv('yunDataN.csv')
print(df_1.shape,df_2.shape,df_4.shape)

(124293, 147) (124293, 92) (124293, 74)


# Mergeing the 4 parts of the datasets with the zeor rate less than 95%

In [24]:
data_parts=[df_1,df_2,df_4]
useful=[] # the column names for each parts that have less than 95% zeros
for ele in data_parts:
  useful_sub=[]
  for col in ele.columns:
    if ele[col].isin([0]).sum()/ele.shape[0]<0.95:
      useful_sub.append(col)
  useful.append(useful_sub)
useful[2]=useful[2][1:]  # drop unuseful column_name in the 4th part

In [25]:
df_1=df_1.loc[:,useful[0]]
df_2=df_2.loc[:,useful[1]]
df_4=df_4.loc[:,useful[2]]

In [26]:
result = pd.concat([df_1,df_2,df_4], axis=1, sort=False) # combine the dateset

In [27]:
result.head()

,_key_client_id,_key_occurreddate_month,Total_incident_count,response_variable,incident_subcatgry_unknown_piv,prgcontct_other_organisation_piv,prgcontct_lwb_worker_piv,prgcontct_both_client_and_carer_piv,prgcontct_department_piv,prgcontct_family_piv,prgcontct_carer_piv,prgcontct_client_piv,prgcontct_meeting_group_of_people_piv_sum_n_days,prgcontct_other_organisation_piv_sum_n_days,prgcontct_educational_institution_piv_sum_n_days,prgcontct_lwb_worker_piv_sum_n_days,prgcontct_both_client_and_carer_piv_sum_n_days,prgcontct_health_professional_piv_sum_n_days,prgcontct_department_piv_sum_n_days,prgcontct_other_stakeholder_piv_sum_n_days,prgcontct_family_piv_sum_n_days,prgcontct_carer_piv_sum_n_days,prgcontct_client_piv_sum_n_days,prgcontct_meeting_group_of_people_piv_days_since,prgcontct_other_organisation_piv_days_since,prgcontct_meeting_lwb_internal_only_piv_days_since,prgcontct_educational_institution_piv_days_since,prgcontct_employer_piv_days_since,prgcontct_lwb_worker_piv_days_since,prgcontct_police_piv_days_since,prgcontct_both_client_and_carer_piv_days_since,prgcontct_health_professional_piv_days_since,prgcontct_department_piv_days_since,prgcontct_other_stakeholder_piv_days_since,prgcontct_family_piv_days_since,prgcontct_carer_piv_days_since,prgcontct_client_piv_days_since,progsubjcat_information_provided_piv,progsubjcat_case_discussion_planning_piv,progsubjcat_i_sight_client_incident_system_generated_piv,...,rowtype_reference_piv_avg_n_days,rowtype_client_address_addition_piv_days_since,rowtype_rp_end_piv_days_since,rowtype_rp_start_piv_days_since,rowtype_progress_note_piv_days_since,rowtype_diagnosis_piv_days_since,rowtype_medictn_start_piv_days_since,rowtype_client_plan_start_piv_days_since,rowtype_client_plan_end_piv_days_since,rowtype_keyworker_change_piv_days_since,rowtype_reference_piv_days_since,rowtype_medictn_end_piv_days_since,rowtype_plcment_end_piv_days_since,rowtype_incident_piv_days_since,rowtype_plcment_start_piv_days_since,rowtype_program_primary_service_type_start_piv_days_since,rowtype_client_alert_start_piv_days_since,rowtype_client_address_addition_piv_lag01,rowtype_rp_end_piv_lag01,rowtype_rp_start_piv_lag01,rowtype_progress_note_piv_lag01,rowtype_diagnosis_piv_lag01,rowtype_medictn_start_piv_lag01,rowtype_client_plan_start_piv_lag01,rowtype_client_plan_end_piv_lag01,rowtype_keyworker_change_piv_lag01,rowtype_reference_piv_lag01,rowtype_medictn_end_piv_lag01,rowtype_plcment_end_piv_lag01,rowtype_incident_piv_lag01,rowtype_plcment_start_piv_lag01,rowtype_program_primary_service_type_start_piv_lag01,rowtype_client_alert_start_piv_lag01,status_placement,status_client_plan,status_rp_mechanical_restraint,status_rp_restricted_access,status_rp_chemical_restraint,status_rp_other,Incident
0,0000D172-EA88-432F-8235-9FAA00D29072,2019-04-01T00:00:00.0,0,0,29,0,0,0,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3650,3650,3650,3650,3650,3650,3650,3650,3650,3650,3650,2868,3650,3650,0,1,0,...,1,603,355,376,890,515,492,761,555,345,2,425,469,703,527,577,488,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,1,2,2,2,0
1,0000D172-EA88-432F-8235-9FAA00D29072,2019-05-01T00:00:00.0,0,0,31,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3650,3650,3650,3650,3650,3650,3650,3650,3650,3650,3650,22,3650,3650,0,0,0,...,1,603,355,376,21,515,492,761,555,345,0,425,469,703,527,577,488,1,1,1,2,1,1,1,1,1,38,1,1,1,1,1,1,2,2,1,2,2,2,0
2,0000D172-EA88-432F-8235-9FAA00D29072,2019-06-01T00:00:00.0,0,0,31,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3650,3650,3650,3650,3650,3650,3650,3650,3650,3650,3650,52,3650,3650,0,0,0,...,1,559,355,376,51,515,492,761,555,345,0,425,469,703,527,577,488,1,1,1,1,1,1,1,1,1,46,1,1,1,1,1,1,2,2,1,2,2,2,0
3,0000D172-EA88-432F-8235-9FAA00D29072,2019-07-01T00:00:00.0,0,0,32,0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3650,3650,3650,3650,3650,3650,3650,3650,3650,3650,3650,83,3650,483,0,1,0,...,1,21,355,376,20,515,492,761,555,345,0,425,469,703,527,577,488,3,1,1,1,1,1,1,1,1,40,1,1,1,1,1,1,2,2,1

### Parse 'Dates' and indexs it

In [28]:
result.to_csv('Dataconcate.csv')

In [29]:
Dataset=pd.read_csv('Dataconcate.csv',parse_dates=['_key_occurreddate_month'],index_col=['_key_client_id','_key_occurreddate_month'])

In [30]:
Dataset.drop(['Unnamed: 0','response_variable'],axis=1).head()

Total_incident_count  ...  Incident
_key_client_id                       _key_occurreddate_month                        ...          
0000D172-EA88-432F-8235-9FAA00D29072 2019-04-01                                  0  ...         0
                                     2019-05-01                                  0  ...         0
                                     2019-06-01                                  0  ...         0
                                     2019-07-01                                  0  ...         0
                                     2019-08-01                                  0  ...         0

[5 rows x 150 columns]

In [31]:
client_id=list(Dataset.index.get_level_values(0)) # the value of '_key_client_id' indexe

### Remove the Useless records

In [32]:
from collections import Counter
occurrences = Counter(client_id)

In [33]:
used_id=[]
usedless_id=[]
for key, value in occurrences.items():
  if value==12:  # number can be changed
    used_id.append(key)
  else:
    usedless_id.append(key)

In [34]:
print(f'Useful client:  {len(used_id):d}\nUseless client: {len(usedless_id):d}')

Useful client:  7380
Useless client: 7007


In [35]:
# remove the rows that the record of client is less than 12 month 
Dataset=Dataset.drop(usedless_id,level=0) 
Dataset.shape

(88560, 152)

In [36]:
Dataset.drop(['Unnamed: 0','response_variable'],axis=1,inplace=True)

## Perparing Dateset for Modelling

### Spliting to Train and Test Set

In [41]:
split=int(0.9*len(used_id))
train_index=used_id[:split] # the clients who will in trainset
test_index=used_id[split:] # the clients who will in testset

# Train dataset
train=Dataset.loc[train_index]
train_x = train.drop('Incident',axis=1)
train_y = train.Incident

# Test Dataset
test=Dataset.loc[test_index]
test_x = test.drop('Incident',axis=1)
test_y = test.Incident

print(train.shape,test.shape)

(79704, 150) (8856, 150)


### Normalize the Data

In [42]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()

train_x = scaler.fit_transform(train_x)

test_x = scaler.transform(test_x)

array([ 0.06627523,  0.00032563,  0.01895286, ..., -0.03468184,
        0.00903265, -0.0753842 ])

###Serializing the Dataset

In [153]:
# 4D output
import numpy as np

def create_dataset(X, y, time_steps,window, number):
  xs, ys,channel = [],[],0
  while channel < number: # ensure all client data change to time series type
    xs_sub, ys_sub =[],[]
    for i in range(window - time_steps):
      shifft= window * channel
      v = X[i+shifft:(i + shifft + time_steps)]
      xs_sub.append(v)
      ys_sub.append(y[i + shifft + time_steps])

    channel+=1
    xs.append(xs_sub)
    ys.append(ys_sub)

  return np.array(xs),np.array(ys)

In [104]:
# 3D output
import numpy as np

def create_dataset(X, y, time_steps,window, number):
  xs, ys,channel = [],[],0
  while channel < number: # ensure all client data change to time series type
    
    for i in range(window - time_steps):
      shifft= window * channel
      v = X[i+shifft:(i + shifft + time_steps)]
      xs.append(v)
      ys.append(y[i + shifft + time_steps])

    channel+=1

  return np.array(xs),np.array(ys)

In [154]:
TIME_STEPS = 4
WINDOW = 12
NUM_TRAIN = len(train_index)
NUM_TEST = len(test_index)

X_train, y_train = create_dataset(train_x,train_y, TIME_STEPS, WINDOW, NUM_TRAIN)
X_test, y_test = create_dataset(test_x, test_y, TIME_STEPS, WINDOW,NUM_TEST)

In [155]:
# [n_samples, (n_sub_sequence),TIME_STEPS, n_features], [n_samples]
print(X_train.shape, y_train.shape)

(6642, 8, 4, 149) (6642, 8)


## LSTM AutoEncoder

In [225]:
# https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/
''' 4D input [n_samples, n_sub_sequence,TIME_STEPS, n_features]'''
def ConvLSMT_autoencode(X):
  model = Sequential()
  model.add(layers.TimeDistributed(layers.Conv1D(filters=16, kernel_size=1, activation='relu'), input_shape=(None, X.shape[2], X.shape[3])))
  model.add(layers.TimeDistributed(layers.MaxPooling1D(pool_size=2)))
  model.add(layers.TimeDistributed(layers.Flatten()))
  model.add(layers.LSTM(32, activation='relu'))
  model.add(layers.RepeatVector(X.shape[2]))
  model.add(layers.LSTM(4, activation='relu', return_sequences=True))
  model.add(layers.Flatten())
  model.add(layers.Dense(8))
  return model

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_78 (TimeDis (None, None, 4, 16)       2400      
_________________________________________________________________
time_distributed_79 (TimeDis (None, None, 2, 16)       0         
_________________________________________________________________
time_distributed_80 (TimeDis (None, None, 32)          0         
_________________________________________________________________
lstm_75 (LSTM)               (None, 32)                8320      
_________________________________________________________________
repeat_vector_28 (RepeatVect (None, 4, 32)             0         
_________________________________________________________________
lstm_76 (LSTM)               (None, 4, 4)              592       
_________________________________________________________________
dense_27 (Dense)             (None, 4, 8)            

In [226]:

'''3D input [n_samples, TIME_STEPS, n_features]'''
def LSTM_autoencoder(X):
    inputs = keras.Input(shape=(X.shape[1], X.shape[2]))
    L1 = layers.LSTM(16, activation='relu', return_sequences=True)(inputs)
    L2 = layers.LSTM(4, activation='relu', return_sequences=False)(L1)
    L4 = layers.RepeatVector(X.shape[1])(L2)
    L5 = layers.LSTM(4, activation='relu', return_sequences=True)(L4)
    L6 = layers.LSTM(16, activation='relu', return_sequences=True)(L5)
    output = layers.Dense(1)(L6)    
    model = keras.Model(inputs=inputs, outputs=output)
    return model

In [227]:
model = ConvLSMT_autoencode(X_train)
model.summary()

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_81 (TimeDis (None, None, 4, 16)       2400      
_________________________________________________________________
time_distributed_82 (TimeDis (None, None, 2, 16)       0         
_________________________________________________________________
time_distributed_83 (TimeDis (None, None, 32)          0         
_________________________________________________________________
lstm_77 (LSTM)               (None, 32)                8320      
_________________________________________________________________
repeat_vector_29 (RepeatVect (None, 4, 32)             0         
_________________________________________________________________
lstm_78 (LSTM)               (None, 4, 4)              592       
_________________________________________________________________
flatten_25 (Flatten)         (None, 16)              

In [228]:
model.compile(loss='mae',optimizer='adam')

In [229]:
history = model.fit(
    X_train,y_train,
    epochs=10,
    batch_size=48,
    validation_split=0.1,shuffle=False)

Epoch 1/10
125/125 [==============================] - 1s 8ms/step - loss: 0.0810 - val_loss: 0.0793
Epoch 2/10
125/125 [==============================] - 1s 6ms/step - loss: 0.0809 - val_loss: 0.0792
Epoch 3/10
125/125 [==============================] - 1s 5ms/step - loss: 0.0809 - val_loss: 0.0792
Epoch 4/10
125/125 [==============================] - 1s 6ms/step - loss: 0.0809 - val_loss: 0.0793
Epoch 5/10
125/125 [==============================] - 1s 5ms/step - loss: 0.0809 - val_loss: 0.0793
Epoch 6/10
125/125 [==============================] - 1s 6ms/step - loss: 0.0809 - val_loss: 0.0793
Epoch 7/10
125/125 [==============================] - 1s 6ms/step - loss: 0.0809 - val_loss: 0.0792
Epoch 8/10
125/125 [==============================] - 1s 6ms/step - loss: 0.0809 - val_loss: 0.0792
Epoch 9/10
125/125 [==============================] - 1s 6ms/step - loss: 0.0809 - val_loss: 0.0793
Epoch 10/10
125/125 [==============================] - 1s 6ms/step - loss: 0.0809 - val_loss: 0.0792

## Find the threshold for dividing positive samples and neigative samples

In [230]:
predict = model.predict(X_test)
predict.shape

(738, 8)

In [231]:
month=['May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
Client=used_id[-y_test.shape[0]:]

preidct_df = pd.DataFrame(predict,columns=month,index=Client,)
True_df=pd.DataFrame(y_test,columns=month,index=Client)

In [232]:
preidct_df.describe()

,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
count,7.380000e+02,7.380000e+02,7.380000e+02,7.380000e+02,7.380000e+02,7.380000e+02,7.380000e+02,7.380000e+02
mean,-7.938240e-05,-6.688498e-05,-1.762673e-04,-6.107589e-05,-1.412406e-04,1.747518e-04,1.255925e-04,-1.150945e-04
std,3.786064e-10,3.858873e-10,1.019325e-09,2.839548e-10,3.057975e-10,6.407185e-10,3.640446e-10,2.257077e-10
min,-7.938278e-05,-6.688536e-05,-1.762683e-04,-6.107561e-05,-1.412403e-04,1.747511e-04,1.255921e-04,-1.150942e-04
25%,-7.938278e-05,-6.688536e-05,-1.762683e-04,-6.107561e-05,-1.412403e-04,1.747511e-04,1.255921e-04,-1.150942e-04
50%,-7.938278e-05,-6.688536e-05,-1.762683e-04,-6.107561e-05,-1.412403e-04,1.747511e-04,1.255921e-04,-1.150942e-04
75%,-7.938278e-05,-6.688536e-05,-1.762683e-04,-6.107561e-05,-1.412403e-04,1.747511e-04,1.255921e-04,-1.150942e-04
max,-7.938278e-05,-6.688536e-05,-1.762683e-04,-6.107561e-05,-1.412403e-04,1.747511e-04,1.255921e-04,-1.150942e-04


In [233]:
preidct_df.head()

,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
DB10AF3E-0F92-E011-A5C6-B6A03279A8B3,-0.000079,-0.000067,-0.000176,-0.000061,-0.000141,0.000175,0.000126,-0.000115
DB1CE471-B20F-434C-B4D4-9F6D00D10976,-0.000079,-0.000067,-0.000176,-0.000061,-0.000141,0.000175,0.000126,-0.000115
F771E77B-341D-4672-B6AD-A43500AB411B,-0.000079,-0.000067,-0.000176,-0.000061,-0.000141,0.000175,0.000126,-0.000115
F77DFB42-B83B-47FD-BBA9-A88D00E983C1,-0.000079,-0.000067,-0.000176,-0.000061,-0.000141,0.000175,0.000126,-0.000115
E9D0885E-57E7-4EE6-A384-A61200D8ABF1,-0.000079,-0.000067,-0.000176,-0.000061,-0.000141,0.000175,0.000126,-0.000115


In [186]:
True_df

,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
DB10AF3E-0F92-E011-A5C6-B6A03279A8B3,0,0,0,0,0,0,0,0
DB1CE471-B20F-434C-B4D4-9F6D00D10976,0,0,0,0,0,0,0,0
F771E77B-341D-4672-B6AD-A43500AB411B,0,0,1,1,0,0,0,0
F77DFB42-B83B-47FD-BBA9-A88D00E983C1,0,0,0,0,1,1,0,0
E9D0885E-57E7-4EE6-A384-A61200D8ABF1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
FF6FAE85-F726-41F9-9AB0-9F6700A5F60D,0,0,1,1,1,1,1,1
FF904024-61CC-44DF-85A4-A2DF00AC73ED,0,1,0,0,0,0,0,0
FFC97BEE-F44B-4BF5-98DA-A82600AB81CA,0,0,0,0,0,0,0,0
FFD43505-079D-465A-BA25-A4AB00EA8F27,0,0,0,0,0,0,0,0


In [168]:
y_test[2]

array([0, 0, 1, 1, 0, 0, 0, 0])